In [4]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from skimage import io
from pywt import dwt2
from sklearn.preprocessing import StandardScaler

def extract_features(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # DCT
    dct_features = cv2.dct(np.float32(gray) / 255.0)
    dct_features = dct_features.flatten()[:100]  # Take the top 100 coefficients
    
    # Wavelet Transform
    coeffs2 = dwt2(gray, 'haar')
    cA, (cH, cV, cD) = coeffs2
    wavelet_features = np.concatenate([cA.flatten(), cH.flatten(), cV.flatten(), cD.flatten()])[:100]
        
    # FFT
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    epsilon = 1e-10  # To avoid log(0)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + epsilon)
    fft_features = magnitude_spectrum.flatten()[:100]  # Take the top 100 coefficients
    
    # Combine features
    features = np.concatenate([dct_features, wavelet_features, fft_features])
    
    # Check for NaN or infinite values
    if np.any(np.isnan(features)) or np.any(np.isinf(features)):
        return None  # Return None for invalid features
    
    return features

# Load dataset
def load_data(base_path):
    features, labels = [], []
    for label in ['real', 'fake']:
        folder_path = os.path.join(base_path, label)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            feature_vector = extract_features(image)
            if feature_vector is not None:  # Only append valid feature vectors
                features.append(feature_vector)
                labels.append(0 if label == 'real' else 1)  # 0 for real, 1 for fake
    return np.array(features), np.array(labels)

# Prepare data
base_path = r"C:\Desktop\ML_Implementation\data(Final_ML)\train"
X, y = load_data(base_path)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
from sklearn.svm import SVC
# Initialize SVM classifier
svm_classifier = SVC(kernel='linear', random_state=42)
# Train the model on the full training set
svm_classifier.fit(X_train, y_train)

# Make predictions
y_pred = svm_classifier.predict(X_test)

# Performance metrics
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.76      0.76       573
           1       0.72      0.70      0.71       500

    accuracy                           0.74      1073
   macro avg       0.73      0.73      0.73      1073
weighted avg       0.74      0.74      0.74      1073

Confusion Matrix:
 [[438 135]
 [149 351]]


In [10]:
from sklearn.tree import DecisionTreeClassifier
# Initialize the Decision Tree Classifier
decision_tree = DecisionTreeClassifier(random_state=42)
# Fit the model on the training data
decision_tree.fit(X_train, y_train)

# Make predictions
y_pred = decision_tree.predict(X_test)

# Performance Metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.70      0.71       573
           1       0.67      0.69      0.68       500

    accuracy                           0.69      1073
   macro avg       0.69      0.69      0.69      1073
weighted avg       0.70      0.69      0.69      1073

[[399 174]
 [154 346]]


In [18]:
from sklearn.ensemble import RandomForestClassifier
# Initialize the Random Forest Classifier with chosen parameters
random_forest = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)

# Fit the model to the training data
random_forest.fit(X_train, y_train)

# Make predictions on the test set
y_pred = random_forest.predict(X_test)

# Performance Metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81       573
           1       0.80      0.76      0.78       500

    accuracy                           0.80      1073
   macro avg       0.80      0.79      0.79      1073
weighted avg       0.80      0.80      0.80      1073

[[476  97]
 [122 378]]


In [24]:
from catboost import CatBoostClassifier
# Initialize the CatBoost Classifier
catboost_model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, random_seed=42, verbose=0)

# Fit the model to the training data
catboost_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = catboost_model.predict(X_test)

# Performance Metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87       573
           1       0.86      0.83      0.85       500

    accuracy                           0.86      1073
   macro avg       0.86      0.86      0.86      1073
weighted avg       0.86      0.86      0.86      1073

[[506  67]
 [ 84 416]]


In [26]:
from sklearn.ensemble import AdaBoostClassifier
# Initialize the base estimator for AdaBoost (e.g., a DecisionTree)
base_estimator = DecisionTreeClassifier(max_depth=1)

# Initialize the AdaBoost Classifier
adaboost_model = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=100, learning_rate=1.0, random_state=42)

# Fit the model to the training data
adaboost_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = adaboost_model.predict(X_test)

# Performance Metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

C:\Users\ru368\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.80      0.83      0.81       573
           1       0.79      0.77      0.78       500

    accuracy                           0.80      1073
   macro avg       0.80      0.80      0.80      1073
weighted avg       0.80      0.80      0.80      1073

[[474  99]
 [117 383]]


In [36]:
from xgboost import XGBClassifier
# Initialize the XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42)

# Fit the model to the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Performance Metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86       573
           1       0.85      0.83      0.84       500

    accuracy                           0.85      1073
   macro avg       0.85      0.85      0.85      1073
weighted avg       0.85      0.85      0.85      1073

[[497  76]
 [ 85 415]]


In [40]:
from sklearn.naive_bayes import GaussianNB
# Initialize the Naive Bayes Classifier
naive_bayes_model = GaussianNB()

# Fit the model to the training data
naive_bayes_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = naive_bayes_model.predict(X_test)

# Performance Metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.73      0.74       573
           1       0.70      0.73      0.72       500

    accuracy                           0.73      1073
   macro avg       0.73      0.73      0.73      1073
weighted avg       0.73      0.73      0.73      1073

[[416 157]
 [134 366]]
